In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, './..')
sys.path.insert(0, '../data')

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec
import proplot as pplt

import numpy as np
import torch
from torchvision import datasets, transforms

from models import model, eval
import plots as pl
from utils import dev, load_data, classification

# Load models

In [ ]:
# load models
model_natural = model.madry_diff()
model_natural.load_state_dict(torch.load('../models/natural_0.pt', map_location=torch.device(dev())))
model_natural.to(dev())
model_natural.eval()

model_robust = model.madry_diff()
model_robust.load_state_dict(torch.load('../models/robust_0.pt', map_location=torch.device(dev())))
model_robust.to(dev())
model_robust.eval()

# Load data

In [ ]:
seed = 0

# load data
data = np.load(f'../data/natural_{seed}.npy', allow_pickle=True).item()
advs = data['advs']
pert_lengths = data['pert_lengths']
classes = data['adv_class']
dirs = data['dirs']
images = data['images']
labels = data['labels']
pert_lengths = data['pert_lengths']

data = np.load(f'../data/robust_{seed}.npy', allow_pickle=True).item()
advs_madry = data['advs']
pert_lengths_madry = data['pert_lengths']
classes_madry = data['adv_class']
dirs_madry = data['dirs']
images_madry = data['images']
labels_madry = data['labels']

# Check seed consistency

In [ ]:
seeds = [0, 1, 2, 3]
model_type = 'natural'
#data = np.load('../data/natural0.npy', allow_pickle=True).item()
#images = data['images']
#labels = data['labels']
#n_advs = np.zeros((3,len(images)))
n_advs = []
for i, model_id in enumerate(seeds):
    # load data
    data = np.load(f'../data/{model_type}_{model_id}.npy', allow_pickle=True).item()
    advs = data['advs']
    pert_lengths = data['pert_lengths']
    classes = data['adv_class']
    dirs = data['dirs']
    images = data['images']
    labels = data['labels']
    pert_lengths = data['pert_lengths']
    n_advs.append(15-np.isnan(pert_lengths).sum(1))
n_advs = np.stack(n_advs, axis=0)
plt.figure(figsize=(15,8))
plt.hist(n_advs.T, alpha=0.5, bins=16, range=(-0.5,15.5))

print(f'mean number of adversarial directions = {np.mean(np.std(n_advs,axis=0))}')

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(n_advs.T)
plt.show()

# Plot grid of adversarial examples

In [ ]:
img_n = 0
fig , ax = pl.plot_advs(advs[img_n], orig=images[img_n], classes=classes[img_n], orig_class=labels[img_n], n=5,vmin=0,vmax=1)
fig.set_figheight(5)
fig.set_figwidth(14)


In [ ]:
fig, ax = plt.subplots(5, 6, squeeze=False, figsize=(9,8))
for j in range(5):
    orig = np.reshape(images[j*50], [28, 28])
    if j == 0:
        ax[j, 0].set_title('original', fontsize=18)
    ax[j, 0].imshow(orig, cmap='gray', vmin=0, vmax=1)
    ax[j, 0].set_xticks([])
    ax[j, 0].set_yticks([])
    ax[j, 0].set_xlabel(str(labels[j*50]), fontdict={'fontsize': 18})

    for i, a in enumerate(advs[j*50,:5]):
        if j==0:
            ax[j, i+1].set_title('Adv. ' + str(i + 1), fontsize=18)
        ax[j, i+1].set_xlabel('\u279E ' + str(int(classes[j*50,i])), fontdict={'fontsize': 18})
        ax[j, i+1].imshow(a.reshape([28, 28]), cmap='gray', vmin=0, vmax=1)
        ax[j, i+1].set_xticks([])
        ax[j, i+1].set_yticks([])
plt.subplots_adjust(hspace=0.3, left=0, right=1, bottom=0.05, top=0.95)
plt.show()

# Plot madry and natural adversarial examples in one figure

In [ ]:
plt.figure(figsize=(12,7))
for i in range(5):
    plt.subplot(3,5,1+i)
    plt.title('Orig. class ' + str(labels[i*50]))
    plt.imshow(np.reshape(images[i*50], [28,28]), cmap='gray', vmin=0, vmax=1)
    plt.xticks([])
    plt.yticks([])
    if i == 0:
        plt.ylabel("Original Image")
    
    plt.subplot(3,5,6+i)
    plt.title('Adv. class ' + str(classes[i*50,0]))
    plt.imshow(np.reshape(advs[i*50,0], [28,28]), cmap='gray', vmin=0, vmax=1)
    plt.xticks([])
    plt.yticks([])
    if i == 0:
        plt.ylabel("Natural CNN")

    plt.subplot(3,5,11+i)
    plt.title('Adv. class ' + str(classes_madry[i*50,0]))
    plt.imshow(np.reshape(advs_madry[i*50,0], [28,28]), cmap='gray', vmin=0, vmax=1)
    plt.xticks([])
    plt.yticks([])
    if i == 0:
        plt.ylabel("Madry CNN")
plt.suptitle('Adversarials of non-robust and robust models')
plt.show()

# Perturbation Length comparison

In [ ]:
# plot with all adversarials included
fig , ax = pl.plot_pert_lengths([pert_lengths, pert_lengths_madry],  n=15, labels=['naturally trained','adversarially trained'], ord=2)
fig.set_figheight(8)
fig.set_figwidth(12)
plt.show()


In [ ]:
# only samples with at least n adversarials included
n=8

p_robust = pert_lengths_madry[np.invert(np.isnan(pert_lengths_madry)).sum(-1)>=n]
p_natural = pert_lengths[np.invert(np.isnan(pert_lengths)).sum(-1)>=n]
        
fig , ax = pl.plot_pert_lengths([p_natural,p_robust],  n=n, labels=['naturally trained','adversarially trained'], ord=2)
fig.set_figheight(5)
fig.set_figwidth(7)
plt.show()

# Plot variation of target classes

In [ ]:
classes_ = classes_madry
pert_lengths_ = pert_lengths_madry

mask_idx = np.invert(np.isnan(pert_lengths)).sum(1)>1
classes_[np.isnan(pert_lengths_)]=np.nan
classes_ = classes_[mask_idx]
labels_ = labels[mask_idx]
fig, ax = pl.plot_var_hist(classes_, labels_, title='Robust CNN', with_colors = True)
fig.set_figheight(5)
fig.set_figwidth(7)
plt.ylim(0,.85)
plt.show()

In [ ]:
mask_idx = np.invert(np.isnan(pert_lengths)).sum(1) > 1
advs_frac_found = np.zeros((2, 10, 10)) # [natural/robust, fraction_label_to, gt_label]
advs_per_class = np.zeros((2, 10))
advs_num_classes = np.zeros((2, 10))
for i, (classes_, pert_lengths_) in enumerate(zip([classes, classes_madry], [pert_lengths, pert_lengths_madry])):
    classes_[np.isnan(pert_lengths_)] = np.nan
    classes_ = classes_[mask_idx]
    labels_ = labels[mask_idx]
    for l in range(10):
        var = np.mean(np.array([(len(np.unique(x[~np.isnan(x)]))-1)/(len(x[~np.isnan(x)])-1) for x in classes_[labels == l]]))
        advs_num_classes[i, l] = var
        u, c = np.unique(classes_[labels_ == l], return_counts=True)
        c = c[~np.isnan(u)]
        u = u[~np.isnan(u)]
        advs_frac_found[i, u.astype(int), l] = c / np.sum(c)
        p = pert_lengths_[labels_ == l]
        advs_per_class[i, l] = np.mean(np.invert(np.isnan(p)).sum(1))
#print('natural data sum per class:\n'+'\n'.join([str(i) + ' : ' + str(advs_frac_found[0, :, l].sum()) for l in range (10)]))
#print('\nrobust data sum per class:\n'+'\n'.join([str(i) + ' : ' + str(advs_frac_found[1, :, l].sum()) for l in range (10)]))

In [ ]:
titles = ['Natural', 'Robust']
colors = ['blue', 'orange', 'green', 'purple', 'red',
          'brown', 'grey', 'pink', 'cyan', 'olive']
plot_labels = [str(i) for i in range(10)]
wspace = ([0.0, 10.0,]*5)[:-1]
fig, axs = pplt.subplots(nrows=2, ncols=10, wspace=wspace)
ax_idx = 0
for l in range(10): # count over each label
    for i in range(2): # show each model per label
        patches, texts = axs[ax_idx].pie(advs_frac_found[i, :, l], cycle=colors, startangle=90, normalize=True)
        axs[ax_idx].format(title=f'{titles[i]} {l:01d}')
        ax_idx += 1
axs[0].legend(patches, labels=plot_labels, loc='upper left', ncols=1)
pplt.show()

In [ ]:
advs_per_class = np.zeros(10)
for l in range(10):
    p = pert_lengths[labels==l]
    advs_per_class[l] = np.mean(np.invert(np.isnan(p)).sum(1))
plt.figure(figsize=(7,5))
plt.bar(range(10), advs_per_class)
plt.xticks(np.arange(0,10))
plt.xlabel('original class label')
plt.ylabel('mean number of directions found')
plt.title('Natural Model')
plt.ylim(0,25)
plt.show()
# print(np.mean(advs_per_class),np.mean(np.invert(np.isnan(pert_lengths)).sum(1)))


# Distance to decision boundary

In [ ]:
n_dim = 8
n_samples = 100
seed = 0
dists = [np.load(f'../data/distance_to_boundary_natural_{seed}.npz')['data'],
         np.load(f'../data/distance_to_boundary_robust_{seed}.npz')['data']]

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
mean_dists = np.nanmean(dists[0],axis=-1)
mask = ~np.isnan(mean_dists)
filtered_data = [d[m] for d, m in zip(mean_dists.T, mask.T)]
ax.boxplot(filtered_data)
ax.plot(np.arange(1,9),np.nanmean(pert_lengths[np.invert(np.isnan(pert_lengths)).sum(-1)>8,:8], axis=0), 'b.')
plt.xlabel('dimension of adversarial space')
plt.ylabel('mean distance to decision boundary')
plt.show()
# dists

In [ ]:
plt.plot(np.arange(2,9),np.mean(np.isnan(dists[0]).sum(-1),axis=0)[1:]/n_samples,'k.')
plt.xlabel('dimension of adversarial space')
plt.ylabel('rate of out of bounds samples')

In [ ]:
colors = ['blue', 'orange']

fig, ax = plt.subplots(figsize=(7,5))
for i, color in enumerate(colors):
    boxprops = dict(color=color, linewidth=1.5, alpha=0.7)
    whiskerprops = dict(color=color, alpha=0.7)
    capprops = dict(color=color, alpha=0.7)
    medianprops = dict(linestyle=None, linewidth=0)
    meanpointprops = dict(marker='o', markeredgecolor='black',
                          markerfacecolor=color)

    mean_dists = np.nanmean(dists[i], axis=-1)
    mask = ~np.isnan(mean_dists)
    filtered_data = [d[m] for d, m in zip(mean_dists.T, mask.T)]
    ax.boxplot(filtered_data, whis=[10,90], showfliers=False, showmeans=False, boxprops=boxprops, 
              whiskerprops=whiskerprops, capprops=capprops, meanprops=meanpointprops,
              medianprops=medianprops)
    if i == 0:
        lengths = pert_lengths
    else:
        lengths = pert_lengths_madry
    x = np.arange(1, 9)
    y = lengths[np.invert(np.isnan(lengths)).sum(-1)>8, :8]
    ax.scatter(x, np.nanmean(y, axis=0), color=color, marker='.')
ax.set_xlabel('dimension of adversarial space')
ax.set_ylabel('mean distance to decision boundary')

plt.show()

In [ ]:
titles = ['Natural', 'Robust']
colors = ['blue', 'orange', 'green', 'purple', 'red',
          'brown', 'grey', 'pink', 'cyan', 'olive']
plot_labels = [str(i) for i in range(10)]

bar_width = 0.35
x = np.arange(len(plot_labels))

fig = plt.figure()
outer_gs = gridspec.GridSpec(nrows=2, ncols=3, figure=fig, hspace=0.5, wspace=0.5)


ax0 = fig.add_subplot(outer_gs[0, 0])
ax0.bar(range(10), advs_per_class[0,:])

ax1 = fig.add_subplot(outer_gs[0, 1])
ax1.bar(range(10), advs_per_class[0,:])

ax_num_classes = fig.add_subplot(outer_gs[1, 0])
ax_num_classes.bar(x-bar_width/2, advs_num_classes[0, :], bar_width, color='blue')
ax_num_classes.bar(x+bar_width/2, advs_num_classes[1, :], bar_width, color='orange')
ax_num_classes.set_xlabel('original class label')
ax_num_classes.set_ylabel('mean number of adversarial classes')
ax_num_classes.set_xticks(range(10))
ax_num_classes.set_ylim([0,1])

ax_dir_found = fig.add_subplot(outer_gs[1, 1])
ax_dir_found.bar(x-bar_width/2, advs_per_class[0, :], bar_width, color='blue')
ax_dir_found.bar(x+bar_width/2, advs_per_class[1, :], bar_width, color='orange')
ax_dir_found.set_xticks(np.arange(0,10), minor=False)
ax_dir_found.set_xticks([], minor=True)
ax_dir_found.set_xlabel('original class label')
ax_dir_found.set_ylabel('mean number of directions found')
ax_dir_found.set_ylim(0,25)

pie_gs = gridspec.GridSpecFromSubplotSpec(nrows=2, ncols=5, subplot_spec=outer_gs[0,2], hspace=-1)
pie_idx = 0
for l in range(10): # label index
    pie_inner_gs = gridspec.GridSpecFromSubplotSpec(nrows=1, ncols=2, subplot_spec=pie_gs[pie_idx], wspace=0.01)
    for i in range(2):
        ax = fig.add_subplot(pie_inner_gs[i])
        patches, texts = ax.pie(advs_frac_found[i, :, l], colors=colors, startangle=90, normalize=True)
        #ax.set_title(f'{l}')
    pie_idx += 1

plt.suptitle("Figure 2")

plt.show()

# Decision Space Visualization

In [ ]:
for img_n in [0,50,100,150,200,250,300,350,400,450]:
    orig = images[img_n]
    adv1 = advs_madry[img_n,0]
    adv2 = advs_madry[img_n,1]
    model_ = model_robust
    pl.plot_dec_space(orig, adv1, adv2, model_, show_legend=True, show_advs=True, overlay_inbounds=True)
    plt.rcParams["figure.figsize"] = (20,3)
    plt.show()